### Download Datasets from Google Drive

In [1]:
%cd ~
!git clone https://github.com/mayocream/tschan

/root
Cloning into 'tschan'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 247 (delta 97), reused 200 (delta 53), pack-reused 0
Receiving objects: 100% (247/247), 6.73 MiB | 22.45 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [2]:
%cd ~/tschan
%pip install gdown
%pip install --upgrade gdown

# acquire the dataset from http://www.manga109.org/ja/download.html
!gdown "1JHpeeBY37E4GWwp3k6T1DSXrN6EOX6QU&confirm=t"
!unzip Manga109.zip
!mv Manga109_released_2021_12_30 datasets/manga109 # keep name consistent with scripts/convert_manga109_to_coco.py
!rm Manga109.zip

/root/tschan
Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: gdown
    Found existing installation: gdown 4.5.1
    Uninstalling gdown-4.5.1:
      Successfully uninstalled gdown-4.5.1
Note: you may need to restart the kernel to use updated packages.
Downloading...
From: https://drive.google.com/uc?id=1JHpeeBY37E4GWwp3k6T1DSXrN6EOX6QU&confirm=t
To: /root/tschan/Manga109.zip
100%|███████████████████████████████████████| 4.10G/4.10G [00:20<00:00, 202MB/s]
Archive:  Manga109.zip
   creating: Manga109_released_2021_12_30/
   creating: Manga109_released_2021_12_30/annotations/
   creating: Manga109_released_2021_12_30/annotations.v2018.05.31/
   creating: Manga109_released_2021_12_30/annotations.v2020.12.18/
   creating: Manga109_released_2021_12_30/images/
  inflating: Manga109_released_2021_12_30/books.txt  
  inflating: Manga109_released_2021_12_30/readme.txt  
   creating: Manga109_released_2021_12_30/images/ARMS/
   creating: Manga109_released_

### Preprecessing datasets

In [3]:
%cd ~/tschan/datasets

%pip install manga109api ultralytics
!python scripts/convert_manga109_to_coco.py \
    --manga109_root_dir manga109 \
    --dataset_version v2021.12.31 \
    --label_filename_prefix manga109_coco \
    --add_manga109_info

# convert coco to yolo format
!python scripts/convert_coco_to_yolo.py

/root/tschan/datasets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.6/615.6 kB 58.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Total books: 109
4val **********************************************************************
processing HealingPlanet
processing LoveHina_vol14
processing SeisinkiVulnus
processing That'sIzumiko
Total images: 419
Total annotations: 9086
15test **********************************************************************
processing Akuhamu
processing BakuretsuKungFuGirl
processing DollGun
processing EvaLady
processing HinagikuKenzan
processing KyokugenCyclone
processing LoveHina_vol01
processing MomoyamaHaikagura
processing TennenSenshiG
processing UchiNoNyan'sDiary
processing UnbalanceTokyo
processing YamatoNoHane
processing YoumaKourin
processing YumeNoKayoiji
processing YumeiroCooking
Total images: 1351
Total annotations: 32642
90train **********************************************************************
processing ARMS
pro

### Training Yolov8

Create a config YAML file inside `datasets/manga109/yolo_labels/`

```
path: /root/tschan/datasets/manga109/yolo_labels  # dataset root dir
train: images/manga109_coco_90train
val: images/manga109_coco_4val
test: images/manga109_coco_15test

# Classes
names:
  0: frame
  1: text
```

In [4]:
%cd ~/tschan/detection
%pip install ultralytics

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='../datasets/manga109/yolo_labels/manga109.yaml', epochs=100, verbose=True, batch=128)
# results = model.train(resume=True)
!cp -rv runs /notebooks/

/root/tschan/detection
Note: you may need to restart the kernel to use updated packages.
    284/300      27.9G     0.3423     0.1869     0.8207       2478        640: 100%|██████████| 138/138 [00:54<00:00,  2.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.00it/s]
                   all        419       9086      0.945       0.94      0.959      0.846

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    285/300      27.1G     0.3438     0.1874     0.8209       2474        640: 100%|██████████| 138/138 [00:54<00:00,  2.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.93it/s]
                   all        419       9086      0.945      0.941      0.959      0.846

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    286/300      29.6G     0.3429     0.1873 

  0%|          | 0.00/49.7M [00:00<?, ?B/s]

Ultralytics YOLOv8.0.190 🚀 Python-3.9.16 torch-1.12.1+cu116 CUDA:0 (NVIDIA A100 80GB PCIe, 81229MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=../datasets/manga109/yolo_labels/manga109.yaml, epochs=300, patience=50, batch=64, imgsz=640, save=True, save_period=100, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnost

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


  0%|          | 0.00/6.23M [00:00<?, ?B/s]

AMP: checks passed ✅
train: Scanning /root/tschan/datasets/manga109/yolo_labels/labels/manga109_coco_90train... 8439 images, 393 backgrounds, 0 corrupt: 100%|██████████| 8832/8832 [00:06<00:00, 1302.23it/s]
train: New cache created: /root/tschan/datasets/manga109/yolo_labels/labels/manga109_coco_90train.cache
val: Scanning /root/tschan/datasets/manga109/yolo_labels/labels/manga109_coco_4val... 399 images, 20 backgrounds, 0 corrupt: 100%|██████████| 419/419 [00:00<00:00, 1303.33it/s]
val: New cache created: /root/tschan/datasets/manga109/yolo_labels/labels/manga109_coco_4val.cache
Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)


Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/300      33.6G     0.8822      1.059      1.074       2764        640: 100%|██████████| 138/138 [01:04<00:00,  2.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.06it/s]
                   all        419       9086      0.909      0.881      0.932      0.776

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/300      29.1G     0.6187     0.4611     0.9048       2673        640: 100%|██████████| 138/138 [00:53<00:00,  2.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.06it/s]
                   all        419       9086      0.904      0.895      0.935      0.775

  

### Inference

In [5]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/train/weights/best.pt')
# run inference on a single image
model.predict('../datasets/bluearchive_comics/images/101.jpg', save=True, imgsz=640, conf=0.5)


image 1/1 /notebooks/detection/../datasets/bluearchive_comics/images/101.jpg: 640x480 4 frames, 8 texts, 7.4ms
Speed: 2.3ms preprocess, 7.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict4


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'frame', 1: 'text'}
 orig_img: array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255,

In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO('../yolov8n.pt')

# Export the model
model.export(format='onnx')

WARNING ⚠️ ../yolov8n.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
  Obtaining dependency information for dill from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.6 MB/s eta 0:00:00ta 0:00:01

requirements: AutoUpdate success ✅ 3.0s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Ultralytics YOLOv8.0.176 🚀 Python-3.11.5 torch-2.0.1 CPU (Apple M2 Pro)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



'../yolov8n.onnx'